In [2]:
from Bio import SeqIO
from ete3 import Tree
from ete3 import TreeNode
from ete3 import NCBITaxa
ncbi = NCBITaxa()
import re
from Bio.SeqUtils import molecular_weight
import pandas
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle

In [3]:
def translateName(sci_name):
    translate_dict = ncbi.get_name_translator([sci_name])
    anID = [v[0] for k,v in translate_dict.items()]
    for ID in anID:
        return str(ID)

In [4]:
def translateID(ID):
    translate_dict = ncbi.get_taxid_translator([ID])
    aName = [v for k,v in translate_dict.items()]
    return str(aName[0])

In [5]:
def digest(sequence, enzyme):
        enzyme_activity_dict = { "t" : r"(?<=K|R)(?!P)" ,
                                 "k" : r"(?<=K)(?!P)" ,
                                 "r" : r"(?<=R)(?!P)" ,
                                 "e" : r"(?<=E)(?!P)" } 
        peps = re.split(enzyme_activity_dict[enzyme], sequence)
        return(peps)

In [6]:
def commonPeps(tree,nodeList,digestionDictionary):
    commonPeps = []
    taxidDict = (ncbi.get_name_translator(nodeList))
    taxidList = []
    for k,v in taxidDict.items():
        taxidList.append(v[0])
    treeValues =[]
    for v in taxidList:
        s = str(v)
        treeVal = tree.search_nodes(name = s)[0]
        treeValues.append(treeVal)
    for node in treeValues:
        if node.is_leaf() == False:
            descendants = node.get_children()
            nodeDict = {int(key.name):[] for key in descendants}
            for node in descendants:
                for n in node:
                    nodeDict[int(node.name)].append(ncbi.get_taxid_translator([int(n.name)]))
            taxNameDict = {}
            for k,v in nodeDict.items():
                newKey = [name for taxid,name in ncbi.get_taxid_translator([k]).items()][0]
                newValue = [[name for taxid,name in value.items()] for value in v]
                taxNameDict[newKey] = newValue
            pepDict = {key:[] for key,value in taxNameDict.items()}
            for k,v in taxNameDict.items():
                for name in v:
                    pepDict[k].append(dig_seq_dic[name[0]])
            sharedPeps = {key:[] for key,value in pepDict.items()}
            for k,v in sharedPeps.items():
                sharedPeps[k].append(set.intersection(*map(set,[i for i in pepDict[k]])))
            commonPeps.append(sharedPeps)
        else:
            pass
            #taxName = ncbi.get_taxid_translator([node.name])
            #key = taxName[int(node.name)]
            #sharedPeps = {key:dig_seq_dic[key]}
            #commonPeps.append(sharedPeps)

    retryList = []
    for element in commonPeps:
        for k,v in element.items():
            treeName1 = translateName(k)
            node1 = tree.search_nodes(name=treeName1)
            for node in node1:
                if node.is_leaf() == False:
                    retryList.append(k)
                else:
                    pass
    return commonPeps,retryList    

In [7]:
sequences = [] 

for record in SeqIO.parse("COL1A1 Final_filtered (1).fasta", "fasta"):
    sequences.append(record.seq)

sequences_clean = [seq.replace("-","") for seq in sequences]
digested_sequences_list = []
for sequence in sequences_clean:
    digested_sequences = (digest(str(sequence),"t"))
    digested_sequences_list.append(digested_sequences) 
    
    
digested_sequences_list_pepmin =[[ pep for pep in lists if len(pep) >= 7 ] for lists in digested_sequences_list]

In [8]:
names =[]

with open ("COL1A1 Final_filtered (1).fasta", "r") as f:
    for line in f:
        if line.startswith(">"):
            names.append(line)
GsNames = []
for n in names:
    GsNames.append(n[n.find("[")+1:n.find("]")])

print(len(GsNames))

185


In [9]:
dig_seq_dic = dict(zip(GsNames,digested_sequences_list_pepmin)) 

dig_seq_dic["Sciurus carolinensis"] = dig_seq_dic.pop("Neosciurus carolinensis")

In [10]:
TaxIDs = []
for n in GsNames:
    TaxIDs.append(int(translateName(n)))

In [11]:
tree = ncbi.get_topology(TaxIDs)
print(tree.get_ascii(attributes=["sci_name"]))


                                                                                           /-Cervus canadensis
                                                                                          |
                                                                                   /Cervidae-Muntiacus reevesi
                                                                                  |       |
                                                                                  |        \-Odocoileus virginianus texanus
                                                                                  |
                                                                             /Pecora             /-Bos taurus
                                                                            |     |       /Bovinae
                                                                            |     |      |       \-Bubalus bubalis
                                                             

In [12]:
with open ("DetailedTreeAlpha1.txt", "w") as f:
    f.write(tree.get_ascii(attributes=["sci_name"]))

In [13]:
firstNodeList = [translateID(int(tree.name))]

In [14]:
sharedPeptidesList = []
sharedPeptides,retryList = commonPeps(tree,firstNodeList,dig_seq_dic)

sharedPeptidesList.append(sharedPeptides)

while len(retryList) > 1:
    sharedPeptides,retryList = commonPeps(tree, retryList, dig_seq_dic)
    sharedPeptidesList.append(sharedPeptides)
    print(sharedPeptides)


[{'Mammalia': [{'GVVGLPGQR', 'GFPGLPGPSGEPGK', 'GFSGLDGAK', 'GANGAPGIAGAPGFPGAR', 'GQAGVMGFPGPK'}], 'Sauria': [{'GVVGLPGQR'}]}, {'Gymnophiona': [{'GVVGLPGQR', 'GRPGPSGPAGGR', 'GNNGEPGSQGHK', 'GSPGEAGRPGEPGLPGAK', 'EGSPGAEGAPGR', 'GFPGSDGAAGPK', 'GEPGPAGVQGPPGPHGEEGK', 'NGDDGEAGKPGRPGER', 'GQPGTAGLPGLK', 'GPAGPPGK', 'GGDGAPGK', 'GDAGPQGSR', 'GQAGVMGFPGPK', 'GFNGLDGAK', 'GSNGSPGNDGAK'}], 'Anura': [{'GQSGVMGFPGPK', 'GFNGLDGAK', 'GDAGPQGSR', 'GEAGEQGER'}]}]
[{'Neobatrachia': [{'GQSGVMGFPGPK', 'GEAGEQGER', 'GLTGSPGSPGPDGK', 'GDAGPQGSR', 'GFNGLDGAK'}], 'Pipidae': [{'GQSGVMGFPGPK', 'GPPGSSGNPGK', 'GATGAPGIAGAPGFPGAR', 'GTVGLPGMR', 'GSPGSDGPAGAPGIPGPQGVAGSR', 'GNNGEPGAQGNK', 'GESGPPGPAGFAGPPGADGQPGAK', 'GAAGEPGK', 'GEAGEQGER', 'GEAGPAGPAGPTGAR', 'EGAPGSEGAPGR', 'NGEDGEAGKPGRPGER', 'GDAGPQGSR', 'GLPGTAGLPGMK', 'GFNGLDGAK', 'GPPGPQGAR'}]}, {'Rhinatrema bivittatum': [{'GEGGPSGPSGPTGAR', 'GAPGPQGAR', 'GNNGEPGSQGHK', 'QYGYDSEK', 'GFPGSDGAAGPK', 'GLTGPIGPPGPAGSPGDK', 'GNDGAPGAAGPSGPTGPTGPPGFPGAVGPK'

[{'Elephantulus edwardii': [{'GDNGAPGAPGSQGAPGLQGMPGER', 'GETGPSGPAGPTGAR', 'GETGPPGPAGFAGPPGADGQPGAK', 'GETGPAGPPGAPGAPGAPGPVGPAGK', 'GNSGEPGAPGNK', 'GFSGLDGAK', 'GPPGPMGPPGLAGPPGESGR', 'GLTGSPGSPGPDGK', 'TGDAGPAGPPGPPGPPGPPGPPSGGYDFSFLPQPPQEK', 'GSEGPQGVR', 'GPPGSAGSPGK', 'GFPGADGIAGPK', 'GETGPAGRPGEVGPPGPPGPAGEK', 'GVPGPPGALGAPGK', 'GVQGPPGPAGPR', 'GSAGPPGATGFPGAAGR', 'GSPGEPGRPGEAGLPGAK', 'GEAGPQGAR', 'QGPSGSSGER', 'NGDDGEAGKPGRPGER', 'TGPSGPAGQDGRPGPPGPPGAR', 'GAAGEPGK', 'GPSGPQGPSGAPGPK', 'GLTGPIGPPGPAGASGDK', 'GAAGLPGPK', 'GFSGLQGPPGPPGSPGEQGPSGASGPAGPR', 'GETGPAGPAGPIGPVGAR', 'GPPGPQGAR', 'GLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPR', 'GEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDIGAPGPSGAR', 'GDAGPAGPK', 'GADGNPGK', 'GFPGLPGPSGEPGK', 'GEPGSPGENGAPGQMGPR', 'GPSGPQGPR', 'GDAGPPGPAGPTGAPGPMGNVGAPGPK', 'GSPGADGPAGSPGTPGPQGIGGQR', 'VGPPGPSGNAGPPGPPGPAGK', 'QMSYGYDEK', 'SSGVSVPGPMGPSGPR', 'GNDGATGAAGPPGPTGPAGPPGFPGAVGAK', 'GLPGTAGLPGMK', 'GQAGVMGFPGPK', 'GRPGAPGPAGAR', 'GVVGLPGQR', 'GEPGPAGVQGPPGPAGEEG

[{'Anas platyrhynchos': [{'GEQGPAGAPGFQGLPGPAGPPGEAGKPGEQGVPGDAGAPGPAGAR', 'GAAGEPGKPGER', 'GETGPAGPPGAPGAPGAPGPVGPAGK', 'GNSGEPGAPGNK', 'GFSGLDGAK', 'EGAPGAEGAPGR', 'GETGPAGPAGPPGPAGAR', 'GPAGPQGPR', 'GDAGPPGPAGPTGAPGPAGAVGAPGPK', 'GLTGSPGSPGPDGK', 'QGPSGSPGER', 'GFPGADGIAGPK', 'GEPGPAGLPGPAGER', 'GATGAPGIAGAPGFPGAR', 'GAAGLPGAK', 'GSAGPPGATGFPGAAGR', 'GEPGPAGPK', 'VGPPGPSGNIGLPGPPGPSGK', 'GSPGADGPIGAPGTPGPQGIAGQR', 'NGDDGEAGKPGRPGER', 'SGGMAVPGPMGPAGPR', 'GPPGPMGPPGLAGPPGEAGR', 'GDAGAPGAPGNQGPPGLQGMPGER', 'GPSGPQGPSGAPGPK', 'GNDGAPGAAGPPGPTGPAGPPGFPGAAGAK', 'GPPGPQGAR', 'GLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPR', 'GSPGPVGPK', 'TGPPGPAGQDGRPGPPGPPGAR', 'GFSGLQGPPGPPGAPGEQGPSGASGPAGPR', 'GFPGLPGPSGEPGK', 'GEPGSPGENGAPGQMGPR', 'GRPGPSGPAGAR', 'GETGDAGAK', 'GPAGPPGK', 'GETGPAGRPGEPGPAGPPGPPGEK', 'QMSYGYDEK', 'GLPGTAGLPGMK', 'GQAGVMGFPGPK', 'GADGAPGK', 'GVVGLPGQR', 'GEPGPAGVQGPPGPAGEEGK', 'GAPGPPGAVGAAGK', 'GLTGPIGPPGPAGAPGDK', 'DGAAGPK', 'TGDVGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEK', 'DGLNGLPGPIGPPG

[{'Cebinae': [{'GETGPAGPAGPVGPVGAR', 'QLSYGYDEK', 'GFSGLDGAK', 'GPAGPQGPR', 'GFPGADGVAGPK', 'GPSGPQGPGGPPGPK', 'GPPGPMGPPGLAGPPGESGR', 'GPPGSAGAPGK', 'GLTGSPGSPGPDGK', 'GSEGPQGVR', 'GSPGADGPAGAPGTPGPQGIAGQR', 'GETGPAGRPGEVGPPGPPGPAGEK', 'GSAGPPGATGFPGAAGR', 'TGDAGPVGPPGPPGPPGPPGPPSAGFDFSFLPQPPQEK', 'NGDDGEAGKPGRPGER', 'GPPGPPGK', 'GNDGATGAAGPPGPTGPAGPAGFPGAVGAK', 'GAAGEPGK', 'GAAGLPGPK', 'GFSGLQGPPGPPGSPGEQGPSGASGPAGPR', 'QGPSGASGER', 'GPPGPQGAR', 'GEPGPVGVQGPPGPAGEEGK', 'STGGISVPGPMGPSGPR', 'TGPPGPAGQDGRPGPPGPPGAR', 'EGAPGAEGSPGR', 'GDAGPAGPK', 'GFPGLPGPSGEPGK', 'GEPGSPGENGAPGQMGPR', 'GESGPAGPPGAPGAPGAPGPVGPAGK', 'VGPPGPSGNAGPPGPPGPAGK', 'GLPGTAGLPGMK', 'GQAGVMGFPGPK', 'GRPGPPGPAGAR', 'GESGPSGPAGPTGAR', 'GVVGLPGQR', 'GADGSPGK', 'GVPGPPGAVGPAGK', 'GEAGPQGPR', 'GEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGAR', 'GEPGPTGLPGPPGER', 'GLTGPIGPPGPAGAPGDK', 'DGLNGLPGPIGPPGPR', 'GEPGPPGPAGAAGPAGNPGADGQPGAK', 'GANGAPGNDGAK', 'GLPGPPGSPGPQGFQGPPGEPGEPGASGPMGPR', 'GNSGEPGAPGSK', 'GDAGAPGAPGSQGAPGLQG

In [15]:
print(retryList)

['Mus']


In [16]:
for x in sharedPeptidesList:
    for y in x:
        for k,v in y.items():
            if len(v[0]) > 0:
                print(k,list(v[0]))

Amniota ['GVVGLPGQR']
Amphibia ['GFNGLDGAK', 'GDAGPQGSR']
Mammalia ['GVVGLPGQR', 'GFPGLPGPSGEPGK', 'GFSGLDGAK', 'GANGAPGIAGAPGFPGAR', 'GQAGVMGFPGPK']
Sauria ['GVVGLPGQR']
Gymnophiona ['GVVGLPGQR', 'GRPGPSGPAGGR', 'GNNGEPGSQGHK', 'GSPGEAGRPGEPGLPGAK', 'EGSPGAEGAPGR', 'GFPGSDGAAGPK', 'GEPGPAGVQGPPGPHGEEGK', 'NGDDGEAGKPGRPGER', 'GQPGTAGLPGLK', 'GPAGPPGK', 'GGDGAPGK', 'GDAGPQGSR', 'GQAGVMGFPGPK', 'GFNGLDGAK', 'GSNGSPGNDGAK']
Anura ['GQSGVMGFPGPK', 'GFNGLDGAK', 'GDAGPQGSR', 'GEAGEQGER']
Neobatrachia ['GQSGVMGFPGPK', 'GEAGEQGER', 'GLTGSPGSPGPDGK', 'GDAGPQGSR', 'GFNGLDGAK']
Pipidae ['GQSGVMGFPGPK', 'GPPGSSGNPGK', 'GATGAPGIAGAPGFPGAR', 'GTVGLPGMR', 'GSPGSDGPAGAPGIPGPQGVAGSR', 'GNNGEPGAQGNK', 'GESGPPGPAGFAGPPGADGQPGAK', 'GAAGEPGK', 'GEAGEQGER', 'GEAGPAGPAGPTGAR', 'EGAPGSEGAPGR', 'NGEDGEAGKPGRPGER', 'GDAGPQGSR', 'GLPGTAGLPGMK', 'GFNGLDGAK', 'GPPGPQGAR']
Rhinatrema bivittatum ['GEGGPSGPSGPTGAR', 'GAPGPQGAR', 'GNNGEPGSQGHK', 'QYGYDSEK', 'GFPGSDGAAGPK', 'GLTGPIGPPGPAGSPGDK', 'GNDGAPGAAGPSGPTGPTGPPG

In [17]:
with open ("finalOutputCOL1A1PepMin.txt","w") as f:
    for x in sharedPeptidesList:
        for y in x:
            for k,v in y.items():
                if len(v[0]) > 0:
                    f.write(k + ": " + str(list(v[0])) + "\n")


In [18]:
finalNames = []
finalPeps = []

for x in sharedPeptidesList: 
    for y in x:
        for k,v in y.items():
                if len(v[0]) > 0:
                    print(k,v)
                    finalNames.append(k)
                    finalPeps.append(v)


Amniota [{'GVVGLPGQR'}]
Amphibia [{'GFNGLDGAK', 'GDAGPQGSR'}]
Mammalia [{'GVVGLPGQR', 'GFPGLPGPSGEPGK', 'GFSGLDGAK', 'GANGAPGIAGAPGFPGAR', 'GQAGVMGFPGPK'}]
Sauria [{'GVVGLPGQR'}]
Gymnophiona [{'GVVGLPGQR', 'GRPGPSGPAGGR', 'GNNGEPGSQGHK', 'GSPGEAGRPGEPGLPGAK', 'EGSPGAEGAPGR', 'GFPGSDGAAGPK', 'GEPGPAGVQGPPGPHGEEGK', 'NGDDGEAGKPGRPGER', 'GQPGTAGLPGLK', 'GPAGPPGK', 'GGDGAPGK', 'GDAGPQGSR', 'GQAGVMGFPGPK', 'GFNGLDGAK', 'GSNGSPGNDGAK'}]
Anura [{'GQSGVMGFPGPK', 'GFNGLDGAK', 'GDAGPQGSR', 'GEAGEQGER'}]
Neobatrachia [{'GQSGVMGFPGPK', 'GEAGEQGER', 'GLTGSPGSPGPDGK', 'GDAGPQGSR', 'GFNGLDGAK'}]
Pipidae [{'GQSGVMGFPGPK', 'GPPGSSGNPGK', 'GATGAPGIAGAPGFPGAR', 'GTVGLPGMR', 'GSPGSDGPAGAPGIPGPQGVAGSR', 'GNNGEPGAQGNK', 'GESGPPGPAGFAGPPGADGQPGAK', 'GAAGEPGK', 'GEAGEQGER', 'GEAGPAGPAGPTGAR', 'EGAPGSEGAPGR', 'NGEDGEAGKPGRPGER', 'GDAGPQGSR', 'GLPGTAGLPGMK', 'GFNGLDGAK', 'GPPGPQGAR'}]
Rhinatrema bivittatum [{'GEGGPSGPSGPTGAR', 'GAPGPQGAR', 'GNNGEPGSQGHK', 'QYGYDSEK', 'GFPGSDGAAGPK', 'GLTGPIGPPGPAGSPGDK', 'GNDGA

In [19]:
dfClassSpecies = []
dfPeptide = []
for k,v in zip(finalNames,finalPeps):
    for i in v[0]:
        print(k,i)
        dfClassSpecies.append(k)
        dfPeptide.append(i)

Amniota GVVGLPGQR
Amphibia GFNGLDGAK
Amphibia GDAGPQGSR
Mammalia GVVGLPGQR
Mammalia GFPGLPGPSGEPGK
Mammalia GFSGLDGAK
Mammalia GANGAPGIAGAPGFPGAR
Mammalia GQAGVMGFPGPK
Sauria GVVGLPGQR
Gymnophiona GVVGLPGQR
Gymnophiona GRPGPSGPAGGR
Gymnophiona GNNGEPGSQGHK
Gymnophiona GSPGEAGRPGEPGLPGAK
Gymnophiona EGSPGAEGAPGR
Gymnophiona GFPGSDGAAGPK
Gymnophiona GEPGPAGVQGPPGPHGEEGK
Gymnophiona NGDDGEAGKPGRPGER
Gymnophiona GQPGTAGLPGLK
Gymnophiona GPAGPPGK
Gymnophiona GGDGAPGK
Gymnophiona GDAGPQGSR
Gymnophiona GQAGVMGFPGPK
Gymnophiona GFNGLDGAK
Gymnophiona GSNGSPGNDGAK
Anura GQSGVMGFPGPK
Anura GFNGLDGAK
Anura GDAGPQGSR
Anura GEAGEQGER
Neobatrachia GQSGVMGFPGPK
Neobatrachia GEAGEQGER
Neobatrachia GLTGSPGSPGPDGK
Neobatrachia GDAGPQGSR
Neobatrachia GFNGLDGAK
Pipidae GQSGVMGFPGPK
Pipidae GPPGSSGNPGK
Pipidae GATGAPGIAGAPGFPGAR
Pipidae GTVGLPGMR
Pipidae GSPGSDGPAGAPGIPGPQGVAGSR
Pipidae GNNGEPGAQGNK
Pipidae GESGPPGPAGFAGPPGADGQPGAK
Pipidae GAAGEPGK
Pipidae GEAGEQGER
Pipidae GEAGPAGPAGPTGAR
Pipidae EGAPGSEGA

Ornithorhynchus anatinus EGAPGSEGAPGR
Ornithorhynchus anatinus GPSGPQGPSGGPGPK
Ornithorhynchus anatinus GSPGSDGPAGAPGTPGPQGIVGQR
Ornithorhynchus anatinus GVPGPPGAVGAAGK
Ornithorhynchus anatinus GAAGLPGAK
Ornithorhynchus anatinus GSAGPPGATGFPGAAGR
Ornithorhynchus anatinus GEAGAQGSR
Ornithorhynchus anatinus GEPGPVGVQGPPGPSGEEGK
Ornithorhynchus anatinus NGDDGEAGKPGRPGER
Ornithorhynchus anatinus GEPGSAGENGAPGQMGPR
Ornithorhynchus anatinus GDAGAPGAPGGQGPPGLQGMPGER
Ornithorhynchus anatinus GFSGLQGPPGPPGSPGEQGPSGASGPAGPR
Ornithorhynchus anatinus GDSGPAGPK
Ornithorhynchus anatinus DGSAGPK
Ornithorhynchus anatinus GPPGPQGAR
Ornithorhynchus anatinus GNDGAPGAAGPPGPTGPAGPPGFPGAVGAK
Ornithorhynchus anatinus QGPSGVSGER
Ornithorhynchus anatinus GEPGPPGPAGAAGPSGNPGSDGQPGAK
Ornithorhynchus anatinus GETGDSGAK
Ornithorhynchus anatinus GFPGLPGPSGEPGK
Ornithorhynchus anatinus GLPGPPGSPGPQGFQGPPGEPGEPGSSGPMGPR
Ornithorhynchus anatinus GPSGPQGPR
Ornithorhynchus anatinus GRPGPSGPAGAR
Ornithorhynchus anatinus 

Bufo bufo GEPGPSGPSGPAGER
Bufo bufo GPSGPQGAR
Bufo bufo SGPPGPPGAR
Bufo bufo SGEPGAAGPPGPSGEK
Bufo bufo GPPGPPGK
Bufo bufo GVAGPPGAVGASGK
Bufo bufo VGPPGPSGNSGPPGPSGPSGK
Bufo bufo GSAGPQGAR
Bufo bufo GNNGEPGASGNK
Bufo bufo GSAGESGRPGEPGLPGAK
Bufo bufo GFNGLPGLPGPPGHSGEQGPSGASGPAGPR
Bufo bufo GEAGEQGER
Bufo bufo SAGISMPGPMGPMGPR
Bufo bufo GRPGPSGPAGAR
Bufo bufo TGPAGAPGQDGR
Bufo bufo GTVGLPGQR
Bufo bufo DGANGLPGPVGPPGPR
Bufo bufo GESGPAGAPGAPGAPGAPGPAGPAGK
Bufo bufo GPPGPSGPPGLAGPPGESGR
Bufo bufo GEQGPAGAPGFQGLPGSPGATGESGKPGEQGVPGDVGPSGPAGAR
Bufo bufo GPPGASGNPGK
Bufo bufo GETGPAGR
Bufo bufo QMSYGYDEK
Bufo bufo GLPGTAGLPGMK
Bufo bufo GSPGSDGPAGAPGIPGPQGIAGTR
Bufo bufo GTAGPPGATGFPGAAGR
Bufo bufo DGEVGAQGPPGPSGPAGER
Bufo bufo GFPGADGASGPK
Bufo bufo GFPGLPGPNGEPGK
Bufo bufo GDTGPAGPK
Bufo bufo GSNGAPGNDGAK
Bufo bufo GETGPAGPAGPAGPAGIR
Bufo bufo GDAGPQGSR
Bufo bufo EGSPGNEGAPGR
Bufo bufo GEPGPAGVQGPPGPSGEEGK
Bufo bufo DGSAGAK
Bufo bufo GSAGLPGAK
Engystomops pustulosus GPPGSSGNPGK
Engystomo

Ranidae GLTGSPGSPGPDGK
Ranidae GSPGESGRPGEPGLPGAK
Ranidae VGPPGPSGNAGPPGPSGPAGK
Ranidae GFPGLPGPTGEPGK
Ranidae NGEDGEAGKPGRPGER
Ranidae GFNGLDGAK
Ranidae GQSGVMGFPGPK
Ranidae GNNGEPGAQGNK
Ranidae QGSSGPSGER
Ranidae GEAGPAGPAGPTGSR
Ranidae DGSNGLPGPIGPPGPR
Ranidae DGSAGPK
Ranidae GEAGEQGER
Ranidae GPPGPQGAR
Ranidae GFPGADGAGGPK
Ranidae GESGPAGSQGPPGPPGEEGK
Ranidae GRPGPSGPAGAR
Ranidae GAGGLPGAK
Ranidae GTVGLPGQR
Ranidae QMSYGYDEK
Ranidae GLPGTAGLPGMK
Ranidae GPDGPQGAR
Ranidae GAIGPPGPQGPR
Ranidae GDAGPQGSR
Ranidae GPPGPSGSPGPQGFQGPPGEPGEPGAAGAMGPR
Ranidae GPPGPSGPPGLAGPPGEPGR
Nanorana parkeri GSAGLPGNK
Nanorana parkeri GPSGPAGAR
Nanorana parkeri GETGPAGRPGEPGASGPPGPSGEK
Nanorana parkeri GSDGAPGK
Nanorana parkeri GPPGSSGSAGK
Nanorana parkeri GEPGNPGENGAPGQVGPR
Nanorana parkeri GPVGSVGPK
Nanorana parkeri GLTGPIGPPGPGGAPGDK
Nanorana parkeri GLTGSPGSPGPDGK
Nanorana parkeri GEAGPAGAAGPTGAR
Nanorana parkeri GEPGPSGPAGFAGPPGSDGQPGAK
Nanorana parkeri TGDVGPAGPPGPPGPPGPPGPPSGGFDFSFMPQPPQEK
Nanor

Euarchontoglires GQAGVMGFPGPK
Dermochelys coriacea GAAGEPGKPGER
Dermochelys coriacea GNSGEPGAPGNK
Dermochelys coriacea GAPGPQGAR
Dermochelys coriacea GFSGLDGAK
Dermochelys coriacea GPAGPQGPR
Dermochelys coriacea TGDVGPAGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEK
Dermochelys coriacea GLTGSPGSPGPDGK
Dermochelys coriacea GVQGPSGPQGPR
Dermochelys coriacea GSPGESGRPGEPGLPGAK
Dermochelys coriacea TGEPGPAGPPGPSGEK
Dermochelys coriacea GEPGPAGLPGPAGER
Dermochelys coriacea TGPTGPAGQDGRPGPAGPPGAR
Dermochelys coriacea GATGAPGIAGAPGFPGAR
Dermochelys coriacea GAAGLPGAR
Dermochelys coriacea GSAGPPGATGFPGAAGR
Dermochelys coriacea GSPGADGAPGAPGIPGPQGIAGQR
Dermochelys coriacea QGPSGSSGER
Dermochelys coriacea GDAGAPGPAGPTGAPGPAGAVGAPGPK
Dermochelys coriacea VGPPGPSGNIGLPGPPGPSGK
Dermochelys coriacea GEPGPVGVQGPPGPSGEEGK
Dermochelys coriacea GDAGAPGAPGNQGPPGLQGMPGER
Dermochelys coriacea GDTGPSGPK
Dermochelys coriacea GPSGPQGPSGAPGPK
Dermochelys coriacea GETGEHGDR
Dermochelys coriacea DGEAGAQGPPGAAGPAGER
Dermochelys 

...

Arvicola amphibius GAPGPAGPK
Arvicola amphibius GNNGAPGNDGAK
Arvicola amphibius TGPPGPAGQDGRPGPPGPPGAR
Arvicola amphibius GDAGPAGPK
Arvicola amphibius GFPGLPGPSGEPGK
Arvicola amphibius GEPGSPGENGAPGQMGPR
Arvicola amphibius GTAGEPGK
Arvicola amphibius GAPGADGPAGSPGTPGPQGIAGQR
Arvicola amphibius GDTGAPGAPGSQGAPGLQGMPGER
Arvicola amphibius QMSYGYDEK
Arvicola amphibius GLPGTAGLPGMK
Arvicola amphibius GNDGATGAAGPPGPTGPTGPPGFPGAAGAK
Arvicola amphibius GQAGVMGFPGPK
Arvicola amphibius GRPGAPGPAGAR
Arvicola amphibius GVVGLPGQR
Arvicola amphibius GEPGPAGVQGPPGPAGEEGK
Arvicola amphibius GADGSPGK
Arvicola amphibius GLTGPIGPPGPAGAPGDK
Arvicola amphibius GEQGPAGSPGFQGLPGPAGPPGEAGKPGEQGVPGDLGAPGPSGAR
Arvicola amphibius GEPGPTGLPGPPGER
Arvicola amphibius DGLNGLPGPIGPPGPR
Arvicola amphibius GEPGPPGPAGAAGPAGNPGADGQPGAK
Arvicola amphibius DGEAGAQGAPGPAGPAGER
Arvicola amphibius VGPPGPSGNAGPPGPPGPVGK
Arvicola amphibius GEPGPPGPAGFAGPPGADGQPGAK
Arvicola amphibius GETGEQGDR
Arvicola amphibius EGSPGAEGSPGR
Ar

Hylobates moloch QLSYGYDEK
Hylobates moloch GETGPAGPPGAPGAPGAPGPVGPAGK
Hylobates moloch GFSGLDGAK
Hylobates moloch GPAGPQGPR
Hylobates moloch GFPGADGVAGPK
Hylobates moloch GPSGPQGPGGPPGPK
Hylobates moloch GPPGPMGPPGLAGPPGESGR
Hylobates moloch GPPGSAGAPGK
Hylobates moloch GLTGSPGSPGPDGK
Hylobates moloch GSEGPQGVR
Hylobates moloch GSPGADGPAGAPGTPGPQGIAGQR
Hylobates moloch GETGPAGRPGEVGPPGPPGPAGEK
Hylobates moloch GDAGPPGPAGPAGPPGPIGNVGAPGAK
Hylobates moloch GVQGPPGPAGPR
Hylobates moloch GSAGPPGATGFPGAAGR
Hylobates moloch DGSPGAK
Hylobates moloch TGDAGPVGPPGPPGPPGPPGPPSAGFDFSFLPQPPQEK
Hylobates moloch NGDDGEAGKPGRPGER
Hylobates moloch GPPGPPGK
Hylobates moloch GAAGEPGK
Hylobates moloch GAAGLPGPK
Hylobates moloch GFSGLQGPPGPPGSPGEQGPSGASGPAGPR
Hylobates moloch QGPSGASGER
Hylobates moloch GPPGPQGAR
Hylobates moloch GEPGPVGVQGPPGPAGEEGK
Hylobates moloch GLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPR
Hylobates moloch STGGISVPGPMGPSGPR
Hylobates moloch TGPPGPAGQDGRPGPPGPPGAR
Hylobates moloch EGAPGAEGSPGR
H

Onychomys torridus GEPGPPGPAGAAGPAGNPGADGQPGAK
Onychomys torridus DGEAGAQGAPGPAGPAGER
Onychomys torridus GEPGPPGPAGFAGPPGADGQPGAK
Onychomys torridus GETGEQGDR
Onychomys torridus EGSPGAEGSPGR
Onychomys torridus SAGVSVPGPMGPSGPR
Onychomys torridus GSPGEAGRPGEAGLPGAK
Onychomys torridus GSPGPAGPK
Onychomys torridus DGSPGPK
Pan troglodytes GETGPAGPAGPVGPVGAR
Pan troglodytes QLSYGYDEK
Pan troglodytes GETGPAGPPGAPGAPGAPGPVGPAGK
Pan troglodytes GFSGLDGAK
Pan troglodytes GPAGPQGPR
Pan troglodytes GFPGADGVAGPK
Pan troglodytes GPSGPQGPGGPPGPK
Pan troglodytes GPPGPMGPPGLAGPPGESGR
Pan troglodytes GPPGSAGAPGK
Pan troglodytes GLTGSPGSPGPDGK
Pan troglodytes GSEGPQGVR
Pan troglodytes GSPGADGPAGAPGTPGPQGIAGQR
Pan troglodytes GETGPAGRPGEVGPPGPPGPAGEK
Pan troglodytes GDAGPPGPAGPAGPPGPIGNVGAPGAK
Pan troglodytes GVQGPPGPAGPR
Pan troglodytes GSAGPPGATGFPGAAGR
Pan troglodytes DGSPGAK
Pan troglodytes TGDAGPVGPPGPPGPPGPPGPPSAGFDFSFLPQPPQEK
Pan troglodytes NGDDGEAGKPGRPGER
Pan troglodytes GPPGPPGK
Pan troglodyte

Macaca nemestrina GPSGPQGPGGPPGPK
Macaca nemestrina GPPGPMGPPGLAGPPGESGR
Macaca nemestrina GLTGSPGSPGPDGK
Macaca nemestrina GSEGPQGVR
Macaca nemestrina GSPGADGPAGAPGTPGPQGIAGQR
Macaca nemestrina GETGPAGRPGEVGPPGPPGPAGEK
Macaca nemestrina GVQGPPGPAGPR
Macaca nemestrina GSAGPPGATGFPGAAGR
Macaca nemestrina NGDDGEAGKPGRPGER
Macaca nemestrina GPPGPPGK
Macaca nemestrina GAAGEPGK
Macaca nemestrina GAAGLPGPK
Macaca nemestrina GDAGPPGPAGPAGPPGPIGNVGAPGPK
Macaca nemestrina GFSGLQGPPGPPGSPGEQGPSGASGPAGPR
Macaca nemestrina QGPSGASGER
Macaca nemestrina TGDAGPVGPPGPPGPPGPPGPPSGGFDFSFLPQPPQEK
Macaca nemestrina GPPGPQGAR
Macaca nemestrina GEPGPVGVQGPPGPAGEEGK
Macaca nemestrina GLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPR
Macaca nemestrina STGGISVPGPMGPSGPR
Macaca nemestrina TGPPGPAGQDGRPGPPGPPGAR
Macaca nemestrina EGAPGAEGSPGR
Macaca nemestrina GDAGPAGPK
Macaca nemestrina GFPGLPGPSGEPGK
Macaca nemestrina GEPGSPGENGAPGQMGPR
Macaca nemestrina VGPPGPSGNAGPPGPPGPAGK
Macaca nemestrina GNDGATGAAGPPGPTGPAGPPGFPGAVGAK
M

Rattus rattus DGEAGAQGAPGPAGPAGER
Rattus rattus VGPPGPSGNAGPPGPPGPVGK
Rattus rattus GSPGADGPAGSPGTPGPQGIAGQR
Rattus rattus GEPGPPGPAGFAGPPGADGQPGAK
Rattus rattus GETGEQGDR
Rattus rattus EGSPGAEGSPGR
Rattus rattus SAGVSVPGPMGPSGPR
Rattus rattus GDAGPPGPAGPAGPPGPIGNVGAPGPK
Rattus rattus GSPGEAGRPGEAGLPGAK
Rattus rattus GSPGPAGPK


In [20]:
mass_charge = []
for i in dfPeptide:
    mass_charge.append(molecular_weight(i,seq_type = "protein"))

In [21]:
containsNorQ = []
for s in dfPeptide:
    if s.find("N") and s.find("Q") == -1:
        print("No")
        containsNorQ.append("No")
    else:
        print("Yes")
        containsNorQ.append("Yes")

Yes
No
Yes
Yes
No
No
No
Yes
Yes
Yes
No
Yes
No
No
No
Yes
Yes
Yes
No
No
Yes
Yes
No
No
Yes
No
Yes
Yes
Yes
Yes
No
Yes
No
Yes
No
No
No
Yes
Yes
Yes
No
Yes
No
No
Yes
Yes
No
No
Yes
No
Yes
Yes
Yes
No
No
No
Yes
Yes
Yes
No
No
Yes
No
Yes
Yes
No
No
No
No
No
Yes
Yes
No
Yes
No
No
No
Yes
No
No
No
No
Yes
Yes
No
No
No
Yes
Yes
No
Yes
Yes
No
Yes
No
Yes
No
Yes
Yes
No
No
No
No
Yes
Yes
Yes
Yes
No
No
Yes
No
No
Yes
No
Yes
No
Yes
Yes
Yes
No
No
No
No
Yes
Yes
Yes
Yes
No
No
No
Yes
No
Yes
No
Yes
Yes
No
Yes
No
No
Yes
No
No
No
No
No
No
No
No
Yes
Yes
Yes
Yes
No
No
Yes
No
Yes
No
No
Yes
Yes
No
Yes
No
Yes
Yes
No
Yes
No
No
No
Yes
No
No
No
No
No
No
No
Yes
No
No
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
No
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
Yes
No
Yes
No
No
No
No
Yes
Yes
No
No
Yes
No
No
Yes
Yes
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
No
Yes
No
Yes
No
No
No
Yes
No
No
No
Yes
Yes
Yes
Yes
No
No
No
No
Yes
No
No
Yes
Yes
No
No
Yes
No
No
No
No
No
Yes
No
No
Yes
No
No
Yes
Yes
No
Yes
Yes
No
No
No
Yes


No
Yes
Yes
No
Yes
Yes
Yes
Yes
No
Yes
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
No
Yes
No
Yes
Yes
Yes
No
Yes
Yes
No
No
Yes
No
Yes
No
No
No
Yes
No
Yes
No
Yes
Yes
No
No
Yes
No
No
Yes
No
No
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
Yes
No
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
No
Yes
Yes
No
Yes
Yes
Yes
No
Yes
Yes
No
Yes
No
Yes
No
No
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
Yes
Yes
No
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
Yes
No
No
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
No
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
No
No
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
No
No
No
No
Yes
No
No
Yes
No
No
No
No
No
No
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
No
No
Yes
No
No
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
No
No
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
No
No
Yes
Yes
No
Yes
No
Yes
Yes
No
No
No
No
Yes
No
Yes
Yes
Yes
No
No
Yes
No
Yes
Yes
No
Yes


No
No
Yes
No
Yes
Yes
Yes
Yes
No
No
Yes
No
No
No
Yes
No
No
No
Yes
Yes
No
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
No
No
Yes
No
No
No
Yes
No
No
No
Yes
No
Yes
Yes
Yes
No
No
Yes
No
No
No
No
No
No
Yes
No
No
No
Yes
No
No
Yes
Yes
No
Yes
No
Yes
Yes
Yes
No
No
No
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
No
No
Yes
No
Yes
No
No
Yes
No
No
No
No
Yes
No
No
Yes
Yes
Yes
No
No
Yes
No
No
No
No
No
No
No
Yes
No
No
Yes
No
No
Yes
No
No
Yes
No
Yes
Yes
Yes
No
No
Yes
No
Yes
No
No
Yes
Yes
No
No
No
Yes
Yes
Yes
No
No
Yes
No
No
No
No
Yes
Yes
No
No
No
Yes
Yes
Yes
No
No
No
Yes
No
No
Yes
No
No
Yes
No
No
Yes
No
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
Yes
No
Yes
No
No
No
No
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
Yes
Yes
No
No
Yes
Yes
No
Yes
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
No
No
Yes
No
Yes
No
No
Yes
No
Yes
No
No
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
Yes
No
No
Yes
No
No
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
No
No
Yes
Y

No
No
Yes
No
No
No
Yes
Yes
No
No
No
Yes
Yes
No
Yes
Yes
Yes
No
No
No
No
Yes
Yes
Yes
No
No
Yes
Yes
Yes
No
No
Yes
Yes
No
No
No
No
Yes
No
No
Yes
Yes
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
No
No
Yes
Yes
No
No
No
Yes
Yes
No
Yes
Yes
Yes
No
No
No
Yes
Yes
Yes
No
Yes
Yes
Yes
No
Yes
No
No
Yes
Yes
No
No
Yes
No
No
Yes
No
No
Yes
No
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
Yes
No
No
No
No
No
Yes
Yes
No
No
Yes
Yes
No
Yes
Yes
Yes
No
No
No
Yes
Yes
No
Yes
Yes
Yes
No
Yes
No
No
Yes
No
No
Yes
No
No
Yes
No
No
Yes
Yes
No
No
No
No
No
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
Yes
No
No
Yes
Yes
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
No
No
Yes
Yes
No
Yes
No
No
Yes
Yes
No
Yes
No
No
Yes
No
Yes
No
Yes
No
Yes
No
Yes
No
Yes
Yes
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
No
Yes
Yes
No
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
No
Yes
No
Yes
No
Yes
No
Yes
No
Yes
No
Yes
Yes
No
No
No
No
No
No
No
No
Yes


No
No
No
No
No
No
Yes
No
No
No
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
Yes
No
Yes
Yes
No
No
Yes
No
No
Yes
Yes
Yes
No
No
Yes
No
Yes
Yes
No
No
No
Yes
No
No
Yes
Yes
No
Yes
Yes
No
No
No
No
No
No
Yes
No
No
Yes
No
Yes
No
No
No
Yes
Yes
No
No
Yes
No
No
Yes
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes
No
No
No
Yes
No
Yes
No
No
Yes
Yes
No
No
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
No
No
Yes
No
No
Yes
No
Yes
No
No
No
Yes
Yes
No
No
Yes
No
No
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
Yes
No
No
Yes
No
No
No
Yes
No
Yes
No
No
Yes
Yes
No
No
No
Yes
No
Yes
Yes
Yes
Yes
No
No
No
No
No
Yes
No
No
No
No
Yes
No
No
No
Yes
Yes
No
No
Yes
No
No
Yes
Yes
No
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
No
No
No
No
No
Yes
No
No
No
No
No
No
No
Yes
No
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
No
No
No
Yes
Yes
Yes
No
No
No
Yes
No
No
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
Yes
Yes
No
Yes
No
Yes
No
No
Yes
No
No
No
No
No
Yes
No
No
No
No
No
Yes
No
No
No
Yes
No
No
Yes
Yes
No
Yes
Yes
Yes
No
Yes
No
Yes
Yes
No
No
Yes
No
No
Yes
No
Yes
Yes
Yes


In [22]:
print(containsNorQ)

['Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'N

In [23]:
d = {"Class" : dfClassSpecies, "Peptide" : dfPeptide, "N/Q" : containsNorQ, "m/z":mass_charge}

In [24]:
df = pandas.DataFrame(d)

In [25]:
df.to_excel("FINAL__TRUEFINALCOL1A1SharedPeps7Minimum.xlsx")

In [26]:
print(tree.get_ascii(attributes=["sci_name"]))


                                                                                           /-Cervus canadensis
                                                                                          |
                                                                                   /Cervidae-Muntiacus reevesi
                                                                                  |       |
                                                                                  |        \-Odocoileus virginianus texanus
                                                                                  |
                                                                             /Pecora             /-Bos taurus
                                                                            |     |       /Bovinae
                                                                            |     |      |       \-Bubalus bubalis
                                                             

In [27]:
print(translateName("Mammalia"))
print(translateName("Archelosauria"))
print(translateName("Episquamata"))

40674
1329799
1329912


In [28]:
Mammalia = tree.search_nodes(name = '40674')[0] 
for x in Mammalia.get_children():
    Mammalia.remove_child(x)


In [29]:
Archelosauria = tree.search_nodes(name = "1329799")[0]
for x in Archelosauria.get_children():
    Archelosauria.remove_child(x)

In [30]:
Episquamata = tree.search_nodes(name= "1329912")[0]
for x in Episquamata.get_children():
    Episquamata.remove_child(x)

In [31]:
print(tree.get_ascii(attributes=["sci_name"]))
with open ("prunedTreeTetrapoda.txt" ,"w") as f:
    f.write(tree.get_ascii(attributes=["sci_name"]))


                /-Mammalia
         /Amniota
        |      |      /-Archelosauria
        |       \Sauria
        |             \-Episquamata
        |
        |                   /-Rhinatrema bivittatum
-Tetrapoda       /Gymnophiona
        |       |           \-Geotrypetes seraphini
        |       |
        |       |                         /-Bufo bufo
        |       |                        |
        |       |                 /Hyloidea-Engystomops pustulosus
         \Amphibia               |       |
                |                |        \-Eleutherodactylus coqui
                |     /Neobatrachia
                |    |           |               /-Lithobates catesbeianus
                |    |           |        /Ranidae
                |    |            \Ranoidea      \-Rana temporaria
                 \Anura                  |
                     |                    \-Nanorana parkeri
                     |
                     |              /-Xenopus laevis
          